This notebook is part of the *orix* documentation https://orix.rtfd.io. Links to the documentation won’t work from the notebook.

# Crystal reference frame

This notebook describes the alignment of crystal reference frames and symmetry
operations in orix.

In [ ]:
%matplotlib inline

from diffpy.structure import Atom, Lattice, Structure
import matplotlib.pyplot as plt

from orix.crystal_map import Phase
from orix.vector import Miller


plt.rcParams.update({
    "figure.figsize": (7, 7),
    "font.size": 20,
    "axes.grid": True,
    "lines.markersize": 10,
    "lines.linewidth": 3,
})

The (direct) Bravais lattice is characterized by basis vectors ($\mathbf{a},
\mathbf{b}, \mathbf{c}$) with unit cell edge lengths ($a$, $b$, $c$) and angles
($\alpha$, $\beta$, $\gamma$). The reciprocal lattice has basis vectors given by

$$
\mathbf{a^*} = \frac{\mathbf{b} \times \mathbf{c}}{\mathbf{a} \cdot (\mathbf{b} \times \mathbf{c})},\:\:
\mathbf{b^*} = \frac{\mathbf{c} \times \mathbf{a}}{\mathbf{a} \cdot (\mathbf{b} \times \mathbf{c})},\:\:
\mathbf{c^*} = \frac{\mathbf{a} \times \mathbf{b}}{\mathbf{a} \cdot (\mathbf{b} \times \mathbf{c})},
$$

with reciprocal lattice parameters ($a^*$, $b^*$, $c^*$) and angles ($\alpha^*$,
$\beta^*$, $\gamma^*$).

Using these two crystallographic lattices, we define a standard Cartesian
(orthonormal) reference frame by the unit vectors ($\mathbf{e_1}, \mathbf{e_2},
\mathbf{e_3}$). In principle, the direct lattice reference frame can be oriented
arbitrarily in the Cartesian reference frame. In `orix` we have chosen

$$
\mathbf{e_1} ||\: \frac{\mathbf{a}}{a},\:\:
\mathbf{e_2} ||\: \mathbf{e_3} \times \mathbf{e_1},\:\:
\mathbf{e_3} ||\: \frac{\mathbf{c^*}}{c^*}.
$$

This alignment is used for example in
<cite data-cite="rowenhorst2015consistent">Rowenhorst et al. (2015)</cite> and
<cite data-cite="degraef2003introduction">De Graef (2003)</cite>, the latter
which was the basis for the *EMsoft* Fortran suite of programs. Another common
option is $\mathbf{e_1} || \mathbf{a^*}/a^*$, $\mathbf{e_2} ||
\mathbf{e_3} \times \mathbf{e_1}$, $\mathbf{e_3} || \mathbf{c}/c$, which is used
for example in
<cite data-cite="britton2016tutorial">Britton et al. (2016)</cite>.

In calculations, it is useful to describe the transformation of the Cartesian
unit column vectors to the coordinates of the direct lattice vectors by the
structure matrix $\mathbf{A}$ (also called the crystal *base*). Given the chosen
alignment of basis vectors with the Cartesian reference frame, $\mathbf{A}$ is
defined as

\begin{equation}
\mathbf{A} = 
\begin{pmatrix}
a & b\cos\gamma & c\cos\beta\\
0 & b\sin\gamma & -c\frac{(\cos\beta\cos\gamma - \cos\alpha)}{\sin\gamma}\\
0 & 0 & \frac{\mathrm{V}}{ab\sin\gamma}
\end{pmatrix},
\end{equation}

where $V$ is the volume of the unit cell.

In `orix`, we use the
[Lattice](https://www.diffpy.org/diffpy.structure/mod_lattice.html#diffpy.structure.lattice.Lattice)
class in `diffpy.structure` to keep track of these properties internally

In [ ]:
lattice = Lattice(1.7, 1.7, 1.4, 90, 90, 120)
lattice

However, by default, `diffpy.structure` uses the other mentioned alignment above,
which is clear from inspecting the structure matrix

In [ ]:
lattice.base

Thus, the alignment is updated whenever a
[Phase](reference.rst#orix.crystal_map.Phase) is created, which bring
together this crystal lattice and a
[Symmetry](reference.rst#orix.quaternion.Symmetry)

In [ ]:
phase = Phase(point_group="321", structure=Structure(lattice=lattice))
phase.structure.lattice.base

<div class="alert alert-info">

Note
    
The lattice is included in a
[Structure](https://www.diffpy.org/diffpy.structure/package.html#diffpy.structure.structure.Structure)
which adds
[atoms](https://www.diffpy.org/diffpy.structure/mod_atom.html#diffpy.structure.atom.Atom)
to the lattice, which is useful when simulating diffraction.

</div>

We can inspect the alignment of the direct and reciprocal lattice basis vectors
with the Cartesian reference frame in the stereographic projection

In [ ]:
abc = Miller(uvw=[[1, 0, 0], [0, 1, 0], [0, 0, 1]], phase=phase)
abcr = Miller(hkl=[[1, 0, 0], [0, 1, 0], [0, 0, 1]], phase=phase)

In [ ]:
fig = abc.scatter(c=["r", "g", "b"], marker="o", return_figure=True, axes_labels=["e1", "e2"])
abcr.scatter(c=["r", "g", "b"], marker="x", s=300, figure=fig)

The crystal symmetry operations are aligned with the direct crystal lattice

In [ ]:
phase.point_group.plot(figure=fig)
fig